In [72]:
from dotenv import load_dotenv
load_dotenv()

True

Gemini 
- 2.0 flash gratuita pero limitada
anthropic(Claude) - gratuita pero limitada
- 
OpenAI api paga

# Componentes

## Prompts

In [1]:
from langchain.prompts import PromptTemplate

promt_template = PromptTemplate.from_template(
    "Tell me a {adejective} joke about {topic}."
)

promt_template.format(
    adejective="funny",
    topic="Python"
)

'Tell me a funny joke about Python.'

## Chain

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | llm
chain.invoke({"input": "Tell me a funny joke about Python."}).content

"Why did the Python say goodbye to the C++ programmer?\n\nBecause he didn't want to get bitten by a class!"

Branching

In [77]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "he product is excellent. I really enjoyed using it and found it very helpful."
result = chain.invoke({"feedback": review})

# Output the result
print(result)


Okay, here's a message to escalate positive feedback to a human agent, designed to highlight the positive nature and encourage further action:

**Option 1 (Short & Sweet):**

> "Escalating positive feedback for review. This customer had a great experience and we should consider sharing this success story/identifying what went well."

**Option 2 (Slightly More Detailed):**

> "Escalating positive customer feedback. The customer reported [briefly mention the positive aspect, e.g., excellent service from agent X, loved the product feature Y].  Recommend reviewing for potential training opportunities or internal recognition."

**Option 3 (Focus on Specific Action):**

> "Positive feedback received! Escalating to identify the agent/team responsible for [mention the positive outcome] so we can acknowledge their contribution and potentially replicate their success."

**Key Considerations When Choosing:**

*   **Your Internal Process:** How does your team typically handle positive feedback? Do

## Memoria 

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente especializado en {abillity}. Puedes responder preguntas sobre {abillity}"),  # Changed from set to tuple
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # Changed from set to tuple
    ]
)
runnable = prompt | model

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
        {"abillity":"ciencia de datos", "input":"Quero aprender ciencia de dados."},
        config={"configurable": {"session_id": "12345"}},
).content

'¡Excelente elección! La ciencia de datos es un campo fascinante y en constante crecimiento. Para ayudarte a empezar, te propongo una hoja de ruta con los pasos clave y recursos útiles:\n\n**1. Fundamentos Matemáticos y Estadísticos:**\n\n*   **Álgebra Lineal:**\n    *   Vectores, matrices, operaciones matriciales, autovalores y autovectores.\n    *   **¿Por qué es importante?**  Modelos de machine learning, procesamiento de imágenes, reducción de dimensionalidad.\n    *   **Recursos:**\n        *   Khan Academy: Álgebra Lineal ([https://es.khanacademy.org/math/linear-algebra](https://es.khanacademy.org/math/linear-algebra))\n        *   Libro: "Linear Algebra and Its Applications" de Gilbert Strang.\n*   **Cálculo:**\n    *   Derivadas, integrales, optimización.\n    *   **¿Por qué es importante?**  Entender cómo funcionan los algoritmos de optimización en machine learning (descenso de gradiente).\n    *   **Recursos:**\n        *   Khan Academy: Cálculo ([https://es.khanacademy.org/m

In [20]:
with_message_history.invoke(
        {"abillity":"ciencia de datos", "input":"Como?"},
        config={"configurable": {"session_id": "12345"}},
).content

'Para empezar tu camino en la ciencia de datos, te sugiero seguir estos pasos iniciales:\n\n**1. Instala Python y un Entorno de Desarrollo:**\n\n*   **Instala Python:**\n    *   Ve a la página oficial de Python: [https://www.python.org/downloads/](https://www.python.org/downloads/)\n    *   Descarga la versión más reciente de Python 3 (por ejemplo, Python 3.12).\n    *   Ejecuta el instalador y asegúrate de marcar la casilla "Add Python to PATH" (esto facilita el uso de Python desde la línea de comandos).\n*   **Elige un Entorno de Desarrollo (IDE):**\n    *   **Anaconda:**  Es una distribución de Python que incluye muchas librerías útiles para ciencia de datos (NumPy, Pandas, Scikit-learn, etc.) y un entorno llamado Jupyter Notebook, que es ideal para aprender y experimentar.  Descárgalo desde [https://www.anaconda.com/products/distribution](https://www.anaconda.com/products/distribution).\n    *   **Visual Studio Code (VS Code):**  Es un editor de código muy popular y versátil.  Pued

## Tools

Solo tiene procesado hasta lo que se entreno, por lo que se pueden usar tools para buscar informacion en otras fuentes 

### Wikipedia

In [29]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print("Nombre de herramienta:", wiki_tool.name) 
print("Descripción:", wiki_tool.description) 
print("Argumentos:", wiki_tool.args) 

Nombre de herramienta: wikipedia
Descripción: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Argumentos: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [31]:
wiki_tool.run("LangChain")  

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Intelligent agent\nSummary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intelligence as the "study and design of intelligent agents," emphasizing that goal-directed behavior is central to intelligence.\nA specialized subset of intelligent agents, agentic AI (also known as an AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, ther

### Servicio Tavily

https://app.tavily.com/home

In [51]:
from langchain_community.tools.tavily_search import TavilySearchResults

searchTavily = TavilySearchResults()
searchTavily.invoke("Que es LangChain?")

[{'title': '¿Qué es LangChain? - Google Cloud',
  'url': 'https://cloud.google.com/use-cases/langchain?hl=es-419',
  'content': '# ¿Qué es LangChain?\n\nLangChain es una plataforma de lenguaje de programación que permite a los desarrolladores construir y conectar modelos para acceder, transformar y compartir datos sin problemas. Ofrece un entorno potente y versátil para el desarrollo de modelos, lo que permite la manipulación de estructuras de datos, la encadenación de modelos y la incorporación de agentes externos como LaMDA. [...] En su núcleo, LangChain aprovecha una arquitectura distribuida que permite un procesamiento eficiente y escalable de datos de lenguaje. Emplea un diseño basado en microservicios, en el que cada cadena se ejecuta como un servicio independiente, lo que facilita la implementación y administración flexibles. Esta arquitectura permite una integración continua con servicios externos, incluidos los LLM y las fuentes de datos basadas en la nube. [...] LangChain fun

In [52]:
#Lista de herramientas
tools = [wiki_tool, searchTavily]

## Agents

Son agentes de software autnomas que interactuan y responder dinamicamente con los usuarios y el entorno
- Capaces dde realizar acciones
- Utilizan y orquestan chains
- Mantiene contexto y memoria

HUB: Almacen de promts de la comunidad

In [53]:
from langchain import hub

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
promt.messages


c:\Users\Socius\Desktop\Langchain\langchain practica\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [54]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

agent_executor.invoke(
    {
        "input": "Hola!",
    })



> Entering new AgentExecutor chain...
Hola! ¿En qué puedo ayudarte hoy?


> Finished chain.


{'input': 'Hola!', 'output': 'Hola! ¿En qué puedo ayudarte hoy?\n'}

In [ ]:
# Usando els ervicio de wikipedia
agent_executor.invoke({"input": "Quiero aprender sobre LangChain.",})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Intelligent agent
Summary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intelligence as the "study and design of intelligent agents," emphasizing that goal-directed behavior is central to intelligence.
A specialized subset of intelligent agents, agentic AI (also known as an AI agent or simply agent), expands this concept by proact

{'input': 'Quiero aprender sobre LangChain.',
 'output': 'LangChain es un marco de software que facilita la integración de modelos de lenguaje grande (LLM) en aplicaciones. Sus casos de uso incluyen análisis y resumen de documentos, chatbots y análisis de código.'}

In [57]:
# Usando el servicio de search
agent_executor.invoke({"input": "Puedes usar la herramienta tavily para buscar cual es la temperatura en buenos aires argentina?",})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'temperatura en buenos aires argentina'}`


[{'title': 'Previsión meteorológica de tres días para Buenos Aires ...', 'url': 'https://www.accuweather.com/es/ar/buenos-aires/7894/weather-forecast/7894', 'content': 'Temperatura. Previsión por horas. 13 57°. rain drop 0 % · 14 60°. rain ... Internacional América del Sur Argentina Ciudad Autónoma de Buenos Aires Buenos Aires', 'score': 0.80070794}, {'title': 'Pronóstico del tiempo por hora para Buenos Aires, Argentina', 'url': 'https://weather.com/es-US/tiempo/horario/l/eb21b6c30fe7eef7bdb5ae103d0669ffbb96ef1e33a2564bf01c340aca0248ed?par=samsung_widget', 'content': 'Pronóstico por horaBuenos Aires, Argentina • A partir de las 7:41 p.m. GMT-03:00 · 1 am. 54°. 24%. Nublado. Sensación térmica54° · 2 am. 54°. 24%. Nublado.', 'score': 0.75421005}, {'title': 'Tiempo actual en Buenos Aires, Ciudad Autónoma de ...', 'url': 'https://www.accuweather.com/es

{'input': 'Puedes usar la herramienta tavily para buscar cual es la temperatura en buenos aires argentina?',
 'output': 'La temperatura en Buenos Aires, Argentina es de 54°F (12°C). La sensación térmica es de 53°F (11.6°C).'}

## RAG

RAG: Retrieval Augmented Generation
Componentes:
- chunks: fragmentos de texto en los que se divide el archivo a procesar
- embeddings: representacion numerica de lo que contiene cada chunk
- consulta: tambien se vuelve una representacion numerica
- retrieval: la consulta se compara con los embeddings y se obtienen los chunks para relevantes

Doc gemini embeddings model: https://ai.google.dev/gemini-api/docs/embeddings

**Primer paso - Preparar los datos**
- generar chunks
- generar embebes y almacenarlos

In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Define the directory containing the text file and the persistent directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "utils", "files", "odyssey.txt")
persistent_directory = os.path.join(current_dir, "utils", "db", "chroma_db")

# Check if the Chroma vector store already exists
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    # Ensure the text file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"The file {file_path} does not exist. Please check the path."
        )

    # Read the text content from the file
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()

    # Split the document into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # Display information about the split documents
    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}\n")

    # Create embeddings
    print("\n--- Creating embeddings ---")
    embeddings = GoogleGenerativeAIEmbeddings(
        model="text-embedding-004"
    )  # Update to a valid embedding model if needed
    print("\n--- Finished creating embeddings ---")

    # Create the vector store and persist it automatically
    print("\n--- Creating vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- Finished creating vector store ---")

else:
    print("Vector store already exists. No need to initialize.")


# Ecosistema

## LangGraph

**LangGraph**
Es una biblioteca para construir app multifactor y con estado utilizando llm
- Puede cordinar multiples chain o agents a travea sw multiples etpadas
- Puede implementar comportamientos, tomar decisiones 

## LangSmith

**LangSmith**: Plataforma unificada para depurar, probar y monitorear aplicaciones de modelos de lenguejas y agentes inteligentes
- gratis para un unico usuario
- https://www.langchain.com/pricing-langsmith

## Hub

**Hub** Prompts de la comunidad 

## LangServe

**LangServe**: Facilita el despliegue y mantenimiento de app, convierte apps de ml en servidores API
- construido con FastAPI
- Se registran todas las entradas y salidas del servidor 

# Modelos

## Antrophic Claude

Anthropic models: 
- https://docs.anthropic.com/en/docs/models-overview 
- Pago

In [ ]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-opus-latest")
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | model
chain.invoke({"input": "Tell me a funny joke about Python."}).content


## OpenAI

- Pago
- Chat Model Documents: https://python.langchain.com/v0.2/docs/integrations/chat/
- OpenAI Chat Model Documents: https://python.langchain.com/v0.2/docs/integrations/chat/openai/

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | model
chain.invoke({"input": "Tell me a funny joke about Python."}).content


## Google - Gemini

- Gratuito con limitaciones 
- https://console.cloud.google.com/gen-app-builder/engines
- https://ai.google.dev/gemini-api/docs/models/gemini

In [75]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
promt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)
chain = promt | llm
chain.invoke({"input": "Tell me a funny joke about Python."}).content

"Why did the Python say goodbye to the C++ programmer?\n\nBecause he didn't want to get bitten by a class!"

## IBM Granite, Flan-T5, Mixtral

- paquete Python langchain-ibm. 
- Clases como WatsonxLLM y ChatWatsonx te permiten incorporar los modelos fundacionales de IBM (como los modelos Granite, Flan-T5, Mixtral, etc.) 
- Modelos de Embeddings: WatsonxEmbeddings te permite generar embeddings de texto usando los modelos de IBM, lo cual es crucial para aplicaciones de RAG (Generación Aumentada por Recuperación) y búsqueda semántica.
Rerankers: WatsonxRerank ofrece capacidades para reordenar los resultados de búsqueda, mejorando la relevancia en los flujos de trabajo de RAG.
- Toolkits: WatsonxToolkit proporciona un conjunto de herramientas diseñadas específicamente para interactuar con los servicios de watsonx dentro de los agentes de LangChain.